<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMSkillsNetworkBD0231ENCoursera2789-2023-01-01">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


## Final Project - Build an ML Pipeline for Airfoil noise prediction


Estimated time needed: **90** minutes


## Scenario


You are a data engineer at an aeronautics consulting company. Your company prides itself in being able to efficiently design airfoils for use in planes and sports cars. Data scientists in your office need to work with different algorithms and data in different formats. While they are good at Machine Learning, they count on you to be able to do ETL jobs and build ML pipelines. In this project you will use the modified version of the NASA Airfoil Self Noise dataset. You will clean this dataset, by dropping the duplicate rows, and removing the rows with null values. You will create an ML pipe line to create a model that will predict the SoundLevel based on all the other columns. You will evaluate the model and towards the end you will persist the model.



## Objectives

In this 4 part assignment you will:

- Part 1 Perform ETL activity
  - Load a csv dataset
  - Remove duplicates if any
  - Drop rows with null values if any
  - Make transformations
  - Store the cleaned data in parquet format
- Part 2 Create a  Machine Learning Pipeline
  - Create a machine learning pipeline for prediction
- Part 3 Evaluate the Model
  - Evaluate the model using relevant metrics
- Part 4 Persist the Model 
  - Save the model for future production use
  - Load and verify the stored model


## Datasets

In this lab you will be using dataset(s):

 - The original dataset can be found here NASA airfoil self noise dataset. https://archive.ics.uci.edu/dataset/291/airfoil+self+noise
 
 - This dataset is licensed under a Creative Commons Attribution 4.0 International (CC BY 4.0) license.


Diagram of an airfoil. - For informational purpose


![Airfoil with flow](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-BD0231EN-Coursera/images/Airfoil_with_flow.png)


Diagram showing the Angle of attack. - For informational purpose


![Airfoil angle of attack](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-BD0231EN-Coursera/images/Airfoil_angle_of_attack.jpg)


## Before you Start


Before you start attempting this project it is highly recommended that you finish the practice project.


## Setup


For this lab, we will be using the following libraries:

*   [`PySpark`](https://spark.apache.org/docs/latest/api/python/index.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMSkillsNetworkBD0231ENCoursera2789-2023-01-01) for connecting to the Spark Cluster


### Installing Required Libraries

Spark Cluster is pre-installed in the Skills Network Labs environment. However, you need libraries like pyspark and findspark to
 connect to this cluster.


The following required libraries are __not__ pre-installed in the Skills Network Labs environment. __You will need to run the following cell__ to install them:


In [3]:
!pip install pyspark==3.1.2 -q
!pip install findspark -q

### Importing Required Libraries

_We recommend you import all required libraries in one place (here):_


In [4]:
# You can also use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

# FindSpark simplifies the process of using Apache Spark with Python

import findspark
findspark.init()

## Part 1 - Perform ETL activity


### Task 1 - Import required libraries


In [5]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import StandardScaler

### Task 2 - Create a spark session


In [6]:
#Create a SparkSession

spark = SparkSession.builder.appName("Noise Prediction").getOrCreate()

23/08/15 12:10:32 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### Task 3 - Load the csv file into a dataframe


Download the data file.

NOTE : Please ensure you use the dataset below and not the original dataset mentioned above.


In [7]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-BD0231EN-Coursera/datasets/NASA_airfoil_noise_raw.csv


--2023-08-15 12:10:43--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-BD0231EN-Coursera/datasets/NASA_airfoil_noise_raw.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60682 (59K) [text/csv]
Saving to: ‘NASA_airfoil_noise_raw.csv.4’

NASA_airfoil_noise_ 100%[===================>]  59.26K  --.-KB/s    in 0.001s  

2023-08-15 12:10:43 (39.1 MB/s) - ‘NASA_airfoil_noise_raw.csv.4’ saved [60682/60682]



Load the dataset into the spark dataframe


In [8]:
# Load the dataset that you have downloaded in the previous task

df = spark.read.csv("NASA_airfoil_noise_raw.csv", header=True, inferSchema=True)


### Task 4 - Print top 5 rows of the dataset


In [9]:
#your code goes here
df.printSchema()
df.show(5)

root
 |-- Frequency: integer (nullable = true)
 |-- AngleOfAttack: double (nullable = true)
 |-- ChordLength: double (nullable = true)
 |-- FreeStreamVelocity: double (nullable = true)
 |-- SuctionSideDisplacement: double (nullable = true)
 |-- SoundLevel: double (nullable = true)

+---------+-------------+-----------+------------------+-----------------------+----------+
|Frequency|AngleOfAttack|ChordLength|FreeStreamVelocity|SuctionSideDisplacement|SoundLevel|
+---------+-------------+-----------+------------------+-----------------------+----------+
|      800|          0.0|     0.3048|              71.3|             0.00266337|   126.201|
|     1000|          0.0|     0.3048|              71.3|             0.00266337|   125.201|
|     1250|          0.0|     0.3048|              71.3|             0.00266337|   125.951|
|     1600|          0.0|     0.3048|              71.3|             0.00266337|   127.591|
|     2000|          0.0|     0.3048|              71.3|             0.00

### Task 6 - Print the total number of rows in the dataset


In [10]:
#your code goes here
rowcount1 = df.count()
print(rowcount1)

1522


### Task 7 - Drop all the duplicate rows from the dataset


In [11]:
df = df.dropDuplicates()


### Task 8 - Print the total number of rows in the dataset


In [12]:
#your code goes here

rowcount2 = df.count()
print(rowcount2)


[Stage 6:======================================================>(199 + 1) / 200]

1503


### Task 9 - Drop all the rows that contain null values from the dataset


In [13]:
df = df.na.drop()


### Task 10 - Print the total number of rows in the dataset


In [14]:
#your code goes here

rowcount3 = df.count()
print(rowcount3)


[Stage 9:=====================================================> (194 + 6) / 200]

1499


### Task 11 - Rename the column "SoundLevel" to "SoundLevelDecibels"Drop


In [15]:
# your code goes here

df = df.withColumnRenamed("SoundLevel", "SoundLevelDecibels")
df.show()

+---------+-------------+-----------+------------------+-----------------------+------------------+
|Frequency|AngleOfAttack|ChordLength|FreeStreamVelocity|SuctionSideDisplacement|SoundLevelDecibels|
+---------+-------------+-----------+------------------+-----------------------+------------------+
|     4000|          3.0|     0.3048|              31.7|             0.00529514|           115.608|
|     3150|          2.0|     0.2286|              31.7|             0.00372371|           121.527|
|     2000|          7.3|     0.2286|              31.7|              0.0132672|           115.309|
|     2000|          5.4|     0.1524|              71.3|             0.00401199|           131.111|
|      500|          9.9|     0.1524|              71.3|              0.0193001|           131.279|
|     1000|         12.6|     0.1524|              71.3|              0.0483159|           122.044|
|    12500|          0.0|     0.0254|              39.6|             4.28464E-4|           129.116|


### Task 12 - Save the dataframe in parquet formant, name the file as "NASA_airfoil_noise_cleaned.parquet"


In [16]:
# your code goes here
df.write.mode("overwrite").parquet("NASA_airfoil_noise_cleaned.parquet")


[Stage 16:>                                                       (0 + 8) / 200]23/08/15 12:11:58 WARN hadoop.MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
23/08/15 12:11:58 WARN hadoop.MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
23/08/15 12:11:59 WARN hadoop.MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers


#### Part 1 - Evaluation



Run the code cell below.<br>
Use the answers here to answer the final evaluation quiz in the next section.<br>
If the code throws up any errors, go back and review the code you have written.


In [17]:
print("Part 1 - Evaluation")

print("Total rows = ", rowcount1)
print("Total rows after dropping duplicate rows = ", rowcount2)
print("Total rows after dropping duplicate rows and rows with null values = ", rowcount3)
print("New column name = ", df.columns[-1])

import os

print("NASA_airfoil_noise_cleaned.parquet exists :", os.path.isdir("NASA_airfoil_noise_cleaned.parquet"))

Part 1 - Evaluation
Total rows =  1522
Total rows after dropping duplicate rows =  1503
Total rows after dropping duplicate rows and rows with null values =  1499
New column name =  SoundLevelDecibels
NASA_airfoil_noise_cleaned.parquet exists : True


## Part - 2 Create a  Machine Learning Pipeline


### Task 1 - Load data from "NASA_airfoil_noise_cleaned.parquet" into a dataframe


In [18]:
#your code goes here

#df = df.repartition(1)
#df.write.mode("overwrite").parquet("NASA_airfoil_noise_cleaned.parquet")
df = spark.read.parquet("NASA_airfoil_noise_cleaned.parquet")
df.show

<bound method DataFrame.show of DataFrame[Frequency: int, AngleOfAttack: double, ChordLength: double, FreeStreamVelocity: double, SuctionSideDisplacement: double, SoundLevelDecibels: double]>

### Task 2 - Print the total number of rows in the dataset


In [19]:
#your code goes here

rowcount4 = df.count()
print(rowcount4)



[Stage 18:=======>                                                  (1 + 7) / 8]

1499


### Task 3 - Define the VectorAssembler pipeline stage


Stage 1 - Assemble the input columns into a single column "features". Use all the columns except SoundLevelDecibels as input features.


In [20]:
#your code goes here
assembler =  VectorAssembler(inputCols=["Frequency", "AngleOfAttack","ChordLength","FreeStreamVelocity","SuctionSideDisplacement"], outputCol="features")



### Task 4 - Define the StandardScaler pipeline stage


Stage 2 - Scale the "features" using standard scaler and store in "scaledFeatures" column


In [21]:
#your code goes here
from pyspark.ml.feature import StandardScaler

scaler = StandardScaler(
    inputCol="features",
    outputCol="scaledFeatures",
    withStd=True,
    withMean=False
)
#scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")


### Task 5 - Define the StandardScaler pipeline stage


Stage 3 - Create a LinearRegression stage to predict "SoundLevelDecibels"


In [22]:
#your code goes here

lr = LinearRegression(
    labelCol="SoundLevelDecibels",
    featuresCol="scaledFeatures",
    predictionCol="prediction"
)
#lr = LinearRegression(featuresCol="scaledFeatures", labelCol="SoundLevelDecibels")


### Task 6 - Build the pipeline


Build a pipeline using the above three stages


In [23]:
#your code goes here

pipeline = Pipeline(stages=[assembler, scaler, lr])


### Task 7 - Split the data


In [24]:
# Split the data into training and testing sets with 70:30 split.
# set the value of seed to 42
# the above step is very important. DO NOT set the value of seed to any other value other than 42.

#your code goes here

(trainingData, testData) = df.randomSplit([0.7, 0.3], seed=42)



### Task 8 - Fit the pipeline


In [25]:
# Fit the pipeline using the training data
# your code goes here

pipeline_model = pipeline.fit(trainingData)


23/08/15 12:14:17 WARN util.Instrumentation: [54d51416] regParam is zero, which might cause numerical instability and overfitting.
[Stage 23:>                                                         (0 + 8) / 8]23/08/15 12:14:20 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/08/15 12:14:20 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
23/08/15 12:14:20 WARN netlib.LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
23/08/15 12:14:20 WARN netlib.LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


#### Part 2 - Evaluation



Run the code cell below.<br>
Use the answers here to answer the final evaluation quiz in the next section.<br>
If the code throws up any errors, go back and review the code you have written.


In [26]:
print("Part 2 - Evaluation")
print("Total rows = ", rowcount4)
ps = [str(x).split("_")[0] for x in pipeline.getStages()]

print("Pipeline Stage 1 = ", ps[0])
print("Pipeline Stage 2 = ", ps[1])
print("Pipeline Stage 3 = ", ps[2])

print("Label column = ", lr.getLabelCol())

Part 2 - Evaluation
Total rows =  1499
Pipeline Stage 1 =  VectorAssembler
Pipeline Stage 2 =  StandardScaler
Pipeline Stage 3 =  LinearRegression
Label column =  SoundLevelDecibels


## Part 3 - Evaluate the Model


### Task 1 - Predict using the model


In [27]:
# Make predictions on testing data
# your code goes here

predictions = pipeline_model.transform(testData)

predictions.select("SoundLevelDecibels", "prediction").show()

[Stage 30:>                                                         (0 + 1) / 1]

+------------------+------------------+
|SoundLevelDecibels|        prediction|
+------------------+------------------+
|           127.315|123.64344009624753|
|           119.975|123.48695788614877|
|           121.783|124.38983849684254|
|           127.224|121.44706993294302|
|           122.229|125.68312652454188|
|           122.754|119.00135887553772|
|           127.564| 126.5260736531985|
|           126.149|124.72369322766609|
|           120.076|129.24665689814083|
|           138.123|130.62951864347926|
|           127.314|127.13089533096246|
|           130.715|125.89163510250594|
|           129.367|129.30423088951827|
|           122.905| 129.9451290107514|
|           127.127|128.10022579415522|
|           127.417|126.28072873047776|
|           128.698|122.06234864411809|
|           131.073|122.38819030163322|
|           135.368|125.76877819819666|
|           124.514|125.43708134590952|
+------------------+------------------+
only showing top 20 rows



### Task 2 - Print the MSE


In [28]:
#your code goes here

evaluator = RegressionEvaluator(labelCol="SoundLevelDecibels", predictionCol="prediction", metricName="mse")

# Calculate the MSE
mse = evaluator.evaluate(predictions)

# Print the MSE
print("Mean Squared Error (MSE):", mse)



[Stage 31:>                                                         (0 + 8) / 8]

Mean Squared Error (MSE): 22.593754071348812


### Task 3 - Print the MAE


In [30]:
#your code goes here
evaluator = RegressionEvaluator(labelCol="SoundLevelDecibels", predictionCol="prediction", metricName="mae")
mae = evaluator.evaluate(predictions)
print(mae)


[Stage 35:>                                                         (0 + 8) / 8]

3.7336902294631287


### Task 4 - Print the R-Squared(R2)


In [31]:
#your code goes here

evaluator = RegressionEvaluator(labelCol="SoundLevelDecibels", predictionCol="prediction", metricName="r2")
r2 = evaluator.evaluate(predictions)
#TODO
print(r2)


[Stage 37:>                                                         (0 + 8) / 8]

0.5426016508689058


#### Part 3 - Evaluation



Run the code cell below.<br>
Use the answers here to answer the final evaluation quiz in the next section.<br>
If the code throws up any errors, go back and review the code you have written.


In [34]:
print("Part 3 - Evaluation")

print("Mean Squared Error = ", round(mse,2))
print("Mean Absolute Error = ", round(mae,2))
print("R Squared = ", round(r2,2))

lrModel = pipeline_model.stages[-1]

print("Intercept = ", round(lrModel.intercept,2))


Part 3 - Evaluation
Mean Squared Error =  22.59
Mean Absolute Error =  3.73
R Squared =  0.54
Intercept =  132.6


## Part 4 - Persist the Model


### Task 1 - Save the model to the path "Final_Project"


In [36]:
# Save the pipeline model as "Final_Project"
# your code goes here
# Remove the existing directory if it exists
!rm -r Final_Project

# Create the directory
!mkdir Final_Project

# Save the model to the specified path
pipeline_model.write().save("Final_Project/pipeline_model")

# Print a message indicating successful save
print("Pipeline model saved to 'Final_Project' directory")

Pipeline model saved to 'Final_Project' directory


### Task 2 - Load the model from the path "Final_Project"


In [37]:
# Load the pipeline model you have created in the previous step
from pyspark.ml.pipeline import PipelineModel
loaded_pipeline_model = PipelineModel.load("Final_Project/pipeline_model")


### Task 3 - Make predictions using the loaded model on the testdata


In [38]:
# Use the loaded pipeline model and make predictions using testingData

loaded_predictions = loaded_pipeline_model.transform(testData)


### Task 4 - Show the predictions


In [39]:
#show top 5 rows from the predections dataframe. Display only the label column and predictions
#your code goes here
loaded_predictions.select("SoundLevelDecibels", "prediction").show(5)

[Stage 59:>                                                         (0 + 1) / 1]

+------------------+------------------+
|SoundLevelDecibels|        prediction|
+------------------+------------------+
|           127.315|123.64344009624753|
|           119.975|123.48695788614877|
|           121.783|124.38983849684254|
|           127.224|121.44706993294302|
|           122.229|125.68312652454188|
+------------------+------------------+
only showing top 5 rows



#### Part 4 - Evaluation



Run the code cell below.<br>
Use the answers here to answer the final evaluation quiz in the next section.<br>
If the code throws up any errors, go back and review the code you have written.


In [47]:
loaded_lr_model = loaded_pipeline_model.stages[-1]

# Retrieve the coefficients for the "Frequency" feature
coeff_frequency = loaded_lr_model.coefficients[0]

# Print the coefficient for the "Frequency" feature
print("Coefficient for Frequency:", coeff_frequency)

coeff_angle_of_attack = loaded_lr_model.coefficients[1]

# Print the coefficient for the "AngleOfAttack" feature
print("Coefficient for AngleOfAttack:", coeff_angle_of_attack)

coeff_chord_length = loaded_lr_model.coefficients[2]

# Print the coefficient for the "ChordLength" feature
print("Coefficient for ChordLength:", coeff_chord_length)

coeff_free_stream_velocity = loaded_lr_model.coefficients[3]

# Print the coefficient for the "FreeStreamVelocity" feature
print("Coefficient for FreeStreamVelocity:", coeff_free_stream_velocity)

# Retrieve the coefficients for the "SuctionSideDisplacement" feature
coeff_suction_side_displacement = loaded_lr_model.coefficients[4]

# Print the coefficient for the "SuctionSideDisplacement" feature
print("Coefficient for SuctionSideDisplacement:", coeff_suction_side_displacement)

print("Part 4 - Evaluation")

loadedmodel = loaded_pipeline_model.stages[-1]
totalstages = len(loaded_pipeline_model.stages)
inputcolumns = loaded_pipeline_model.stages[0].getInputCols()

print("Number of stages in the pipeline = ", totalstages)
for i,j in zip(inputcolumns, loadedmodel.coefficients):
    print(f"Coefficient for {i} is {round(j,4)}")

Coefficient for Frequency: -3.9728079740154767
Coefficient for AngleOfAttack: -2.477510117497856
Coefficient for ChordLength: -3.3817506717710835
Coefficient for FreeStreamVelocity: 1.5789334478964052
Coefficient for SuctionSideDisplacement: -1.6465101501795425
Part 4 - Evaluation
Number of stages in the pipeline =  3
Coefficient for Frequency is -3.9728
Coefficient for AngleOfAttack is -2.4775
Coefficient for ChordLength is -3.3818
Coefficient for FreeStreamVelocity is 1.5789
Coefficient for SuctionSideDisplacement is -1.6465


### Stop Spark Session


In [43]:
spark.stop()

## Authors


[Ramesh Sannareddy](https://www.linkedin.com/in/rsannareddy/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMBD0231ENSkillsNetwork866-2023-01-01)


### Other Contributors


## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2023-05-26|0.1|Ramesh Sannareddy|Initial Version Created|


Copyright © 2023 IBM Corporation. All rights reserved.
